# RANDOM FOREST by gaurav

### Load libraries and data

In [1]:
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# check missing values per column
train.isnull().sum(axis=0)/train.shape[0]

ID             0.000000
datetime       0.000000
siteid         0.099896
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      0.050118
devid          0.149969
click          0.000000
dtype: float64

In [4]:
train.click.value_counts()/train.shape[0]

0    0.963979
1    0.036021
Name: click, dtype: float64

In [5]:
train[train.click==1].shape

(437214, 10)

### Clean Data and Create Features

In [6]:
train['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None", inplace=True)
test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None", inplace=True)
test['devid'].fillna("None", inplace=True)

In [7]:
# set datatime
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [8]:
# create datetime variable
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

test['tweekday'] = test['datetime'].dt.weekday
test['thour'] = test['datetime'].dt.hour
test['tminute'] = test['datetime'].dt.minute

In [9]:
cols = ['siteid','offerid','category','merchant']

for x in cols:
    train[x] = train[x].astype('object')
    test[x] = test[x].astype('object')

In [10]:
cat_cols = cols + ['countrycode','browserid','devid']

In [11]:
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))

In [12]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,tweekday,thour,tminute
0,IDsrk7SoW,2017-01-14 09:42:09,128865,784773,48,127,4,2,2,0,5,9,42
1,IDmMSxHur,2017-01-18 17:50:53,142053,157563,59,65,1,8,0,0,2,17,50
2,IDVLNN0Ut,2017-01-11 12:46:49,2618,458279,69,15,0,1,2,0,2,12,46
3,ID32T6wwQ,2017-01-17 10:18:43,243406,345067,117,507,2,2,1,0,1,10,18
4,IDqUShzMg,2017-01-14 16:02:33,154278,417948,36,276,3,8,0,0,5,16,2


### Model Training

In [15]:
cols_to_use = list(set(train.columns) - set(['ID','datetime','click']))

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=20, random_state=42)
a = 0
for i, j in skf.split(train[cols_to_use], train['click']):
    a = j
len(a)
# train.loc[a].to_csv("small_train.csv")


606889

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[a][cols_to_use], train.loc[a]['click'], test_size = 0.5, random_state=42)

# For trial

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
t1 = time.time()
dict3 = {}
dict4 = {}
for i in range(250, 1000, 15):
    
    print("Model with ", i, " max_features ")
    model_mod = RandomForestClassifier(n_estimators=i, min_samples_leaf=20, verbose=1, oob_score=True, n_jobs=4, random_state=42)#lets use the fucing cores
    start = time.time()
    model_mod.fit(X_train[cols_to_use], y_train)
    print("time to fit: ", time.time()-start)
    start = time.time()
    pred = model_mod.predict(X_test)
    print("time to predict: ", time.time()-start ,"\n")
    print("training score: ", roc_auc_score(y_train, model_mod.predict(X_train)))
    print("crossval score: ", roc_auc_score(y_test, pred))
    print("oob score: ", model_mod.oob_score_)
    dict3[i]=roc_auc_score(y_test, pred)
    dict4[i]=model_mod.oob_score_
    print("Sort of truth table bouy")
    pred = pd.DataFrame(pred)
    print(pred[0].value_counts())
    y_test = pd.Series(y_test)
    print(y_test.value_counts())
    print("Done :: ", i)
print("total time ", time.time()-t1)



Model with  250  max_features 


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   25.1s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:   32.1s finished


time to fit:  40.175071239471436


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    2.6s finished


time to predict:  3.078465700149536 



...

time to fit:  168.2838909626007


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done 985 out of 985 | elapsed:   10.5s finished


time to predict:  11.83913803100586 



[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 985 out of 985 | elapsed:   10.7s finished


training score:  0.819835071851
crossval score:  0.812518885455
oob score:  0.978150828489
Sort of truth table bouy
0    293836
1      9609
Name: 0, dtype: int64
0    292472
1     10973
Name: click, dtype: int64
Done ::  985
total time  6185.811676979065


In [28]:
import operator
max(dict3.items(), key=operator.itemgetter(1))

(805, 0.81306055344133166)

In [34]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(dict3.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_x)

[(805, 0.81306055344133166), (445, 0.8129859244399793), (760, 0.81291701600919952), (325, 0.81286290179216847), (430, 0.81284922526928749), (835, 0.81284297888233947), (820, 0.81284126931697931), (865, 0.81283272149017882), (790, 0.81282759279409855), (775, 0.81281220670585763), (460, 0.81280878757513741), (850, 0.8127854455345882), (340, 0.81277689770778772), (535, 0.81275013653651829), (895, 0.81274329827507785), (910, 0.81274329827507785), (505, 0.81273816957899747), (355, 0.8127330408829172), (880, 0.81270286058092756), (970, 0.8126926031887669), (475, 0.81268576492732647), (385, 0.81268405536196642), (415, 0.8126806362312462), (940, 0.81265900375605704), (520, 0.81265216549461672), (925, 0.81260659910438615), (685, 0.8126048895390261), (715, 0.81260147040830588), (745, 0.8125963417122255), (655, 0.81259463214686534), (670, 0.81259121301614512), (370, 0.81258779388542501), (400, 0.81258779388542501), (700, 0.8125610327141557), (730, 0.8125610327141557), (490, 0.81255932314879553), 

In [35]:
sorted_x = sorted(dict4.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_x)

[(550, 0.97819037450073165), (565, 0.97819037450073165), (295, 0.97818048799778545), (535, 0.97818048799778545), (430, 0.97817719249680335), (250, 0.97817389699582136), (265, 0.97817389699582136), (520, 0.97817389699582136), (325, 0.97817060149483925), (685, 0.97817060149483925), (310, 0.97816730599385715), (595, 0.97816730599385715), (670, 0.97816730599385715), (700, 0.97816730599385715), (805, 0.97816730599385715), (910, 0.97816730599385715), (655, 0.97816401049287516), (940, 0.97816401049287516), (400, 0.97816071499189305), (580, 0.97816071499189305), (340, 0.97815741949091106), (730, 0.97815741949091106), (895, 0.97815741949091106), (925, 0.97815741949091106), (970, 0.97815741949091106), (355, 0.97815412398992896), (370, 0.97815412398992896), (505, 0.97815412398992896), (625, 0.97815412398992896), (715, 0.97815412398992896), (745, 0.97815412398992896), (475, 0.97815082848894686), (640, 0.97815082848894686), (835, 0.97815082848894686), (985, 0.97815082848894686), (460, 0.97814753298

    400, njob=-1
    training score:  1.0
    crossval score:  0.799803514688
    oob score:  0.978233216013

    0    294387
    1      9058
    Name: 0, dtype: int64
    0    292421
    1     11024
    Name: click, dtype: int64

    600, njob=-1
    worse performance

    470, 510 looks good to me (510 !!!)

    d1,2 : nestimators
    d 5,6 :
    d 3, 4 ; min sample split

# MODEL 1
    test oob score:  0.922385828199
    public score : 0.67484
    dont change this one now, add more cells
    imbalanced data treated : yes.
    n_estimators=500, min_samples_leaf=20, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 7
    test oob score: 0.925401618252
    public score : ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=500, min_samples_leaf=10, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 8
    test oob score: 0.933106663284
    public score :  ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=500, min_samples_leaf=10, verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 9
    test oob score: 0.929116530453
    public score :  ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=500, min_samples_split=50, verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 2
    test oob score:  0.929364126495
    public score : 0.67450 ;     imbalanced data treated : yes.
    crossval score:  0.814501582275; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=470, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 3
    test oob score: 0.923033361724
    public score :   ;     imbalanced data treated : yes.
    crossval score:  0.812722783491; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=470, min_samples_leaf=20,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 4
    test oob score: 0.922004965297
    public score : ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=470, min_samples_leaf=20, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 5
    test oob score:  0.925431065727
    public score : ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=470, min_samples_leaf=10, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42

# MODEL 6
    test oob score: 0.933117989236
    public score :  0.67483;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=470, min_samples_leaf=10, verbose=1, oob_score=True, n_jobs=-1, random_state=42

## models to try 
-  min_samples_leaf=20, 10
- both on 470
- both on 500, 510
- find new minima on n_estimators for ensemble
    - 

Imbalance treat

In [29]:
df_ones = train[train.click==1]
df_zero = train[train.click==0]
rows = np.random.choice(df_zero.index.values, 430000)
print(df_ones.shape[0])
new_df = df_ones.append(df_zero.loc[rows])
print(new_df.shape[0])
rows = np.random.choice(new_df.index.values, new_df.shape[0])
new_df = new_df.loc[rows]
# X_train, X_test, y_train, y_test = train_test_split(new_df[cols_to_use], new_df['click'], test_size = 0.5, random_state=42)

437214
867214


In [30]:
new_df.to_csv("balTrain.csv")

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
t1 = time.time()
dict5 = {}
dict6 = {}
for i in [20]:#, 4, 5, 10, 20, 50, 60, 70, 80, 100, 300, 500]:
    
    print("Model with ", i, " n estimators")
    model_mod = RandomForestClassifier(n_estimators=500, min_samples_leaf=20, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42)#lets use the fucing cores
    start = time.time()
    model_mod.fit(new_df[cols_to_use], new_df['click'])
    print("time to fit: ", time.time()-start)
    start = time.time()
#     pred = model_mod.predict(X_test)
    print("time to predict: ", time.time()-start ,"\n")
    print("training score: ", roc_auc_score(y_train, model_mod.predict(X_train)))
#     print("crossval score: ", roc_auc_score(y_test, pred))
    print("oob score: ", model_mod.oob_score_)
#     dict3[i]=roc_auc_score(y_test, pred)
#     dict4[i]=model_mod.oob_score_
    print("Done :: ", i)
print("total time ", time.time()-t1)

Model with  20  n estimators


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  4.4min finished


time to fit:  327.42327094078064
time to predict:  0.0 



[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.6s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    3.9s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    4.6s finished


training score:  0.914238498067
oob score:  0.92205366689
Done ::  20
total time  332.85011863708496


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
t1 = time.time()
dict5 = {}
dict6 = {}
for i in [20]:#, 4, 5, 10, 20, 50, 60, 70, 80, 100, 300, 500]:, min_samples_split=50
    
    print("Model with ", i, " n estimators")
    model_mod = RandomForestClassifier(n_estimators=470, min_samples_leaf=10, verbose=1, oob_score=True, n_jobs=-1, random_state=42)#lets use the fucing cores
    start = time.time()
    model_mod.fit(new_df[cols_to_use], new_df['click'])
    print("time to fit: ", time.time()-start)
    start = time.time()
#     pred = model_mod.predict(X_test)
    print("time to predict: ", time.time()-start ,"\n")
    print("training score: ", roc_auc_score(y_train, model_mod.predict(X_train)))
#     print("crossval score: ", roc_auc_score(y_test, pred))
    print("oob score: ", model_mod.oob_score_)
#     dict3[i]=roc_auc_score(y_test, pred)
#     dict4[i]=model_mod.oob_score_
    print("Done :: ", i)
print("total time ", time.time()-t1)

Model with  20  n estimators


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 470 out of 470 | elapsed:  4.3min finished


time to fit:  319.32347989082336
time to predict:  0.0 



[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done 470 out of 470 | elapsed:    4.9s finished


training score:  0.91970711088
oob score:  0.933117989236
Done ::  20
total time  325.21167731285095


In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
t1 = time.time()
dict5 = {}
dict6 = {}
for i in [20]:#, 4, 5, 10, 20, 50, 60, 70, 80, 100, 300, 500]:, min_samples_split=50
    
    print("Model with ", i, " n estimators")
    model_mod = RandomForestClassifier(n_estimators=470, min_samples_split=50,  verbose=1, oob_score=True, n_jobs=-1, random_state=42)#lets use the fucing cores
    start = time.time()
    model_mod.fit(new_df[cols_to_use], new_df['click'])
    print("time to fit: ", time.time()-start)
    start = time.time()
#     pred = model_mod.predict(X_test)
    print("time to predict: ", time.time()-start ,"\n")
    print("training score: ", roc_auc_score(y_train, model_mod.predict(X_train)))
#     print("crossval score: ", roc_auc_score(y_test, pred))
    print("oob score: ", model_mod.oob_score_)
#     dict3[i]=roc_auc_score(y_test, pred)
#     dict4[i]=model_mod.oob_score_
    print("Done :: ", i)
print("total time ", time.time()-t1)

Model with  20  n estimators


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 470 out of 470 | elapsed:  4.2min finished


time to fit:  315.5517942905426
time to predict:  0.0 



[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    1.7s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    4.4s
[Parallel(n_jobs=12)]: Done 470 out of 470 | elapsed:    4.8s finished


training score:  0.917275777748
oob score:  0.929088215574
Done ::  20
total time  321.21484112739563


# MODEL 8
    test oob score: 0.933106663284
    public score :  ;     imbalanced data treated : yes.
    crossval score: ; on untreated sample of data random state = 42
    dont change this one now, add more cells
    n_estimators=500, min_samples_leaf=10, verbose=1, oob_score=True, n_jobs=-1, random_state=42

In [46]:
pred_1 = model_mod.predict_proba(test[cols_to_use])
pred_1 = pd.DataFrame(pred)

[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    3.6s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   20.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   48.3s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:  1.0min finished


In [48]:
pred_2 = model_mod.predict_proba(test[cols_to_use])
pred_2 = pd.DataFrame(pred)

[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    4.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   23.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   54.7s
[Parallel(n_jobs=12)]: Done 470 out of 470 | elapsed:  1.0min finished


In [50]:
pred_3 = model_mod.predict_proba(test[cols_to_use])
pred_3 = pd.DataFrame(pred)

[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    3.7s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   21.4s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   52.8s
[Parallel(n_jobs=12)]: Done 470 out of 470 | elapsed:   57.8s finished


In [51]:
pred = (pred_1+pred_2+pred_3)/3

In [52]:

sub = pd.DataFrame({'ID':test['ID'], 'click':pred[1]})
sub.to_csv('bal_ens_1.csv', index=False)

In [44]:
test.to_csv("test_modded.csv", index=False)